In [ ]:
# 1. COLLEGARE GOOGLE DRIVE
from google.colab import drive
import os
import pandas as pd
import numpy as np

# Questo comando ti chiederà l'autorizzazione per accedere al Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# --- CONFIGURAZIONE PERCORSI PER COLAB ---
# Base directory nel tuo Drive
BASE_DIR = "/content/drive/MyDrive/train/"

# Verifica che i percorsi siano giusti
print(f"Contenuto della cartella train: {os.listdir(BASE_DIR)}")

# Impostiamo i path specifici
# Assumo che i CSV siano dentro una sottocartella 'data'.
# Se sono direttamente in 'train', cancella la parte 'data/'
PATH_TRAIN_DATA = os.path.join(BASE_DIR, "data/")
PATH_LABELS = os.path.join(BASE_DIR, "labels.xlsx")

In [ ]:
# --- INIZIO CODICE DI ELABORAZIONE ---

def process_labels_and_targets(path_labels):
    """
    Legge l'Excel, mantiene tutte le colonne originali e
    aggiunge le colonne Target calcolate per i 5 Task.
    """
    print(f"Lettura file labels: {path_labels}")

    # header=1 per saltare la riga di raggruppamento
    df = pd.read_excel(path_labels, header=1)

    # Rinominiamo le prime colonne essenziali
    df.rename(columns={
        df.columns[0]: 'Case',
        df.columns[1]: 'Spacecraft',
        df.columns[2]: 'Condition'
    }, inplace=True)

    # Inizializziamo le colonne Target
    df['Target_Task1'] = 0       # 0:Normal, 1:Abnormal
    df['Target_Task2'] = 0       # 0:Normal, 1:Unknown, 2:Bubble, 3:Valve
    df['Target_Task3'] = 0       # 0:None, 1-8:Bubble Loc
    df['Target_Task4'] = 0       # 0:None, 1-4:Valve Loc
    df['Target_Task5'] = 100.0   # Ratio

    # Mapping per Location
    bubble_cols = ['BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BV1']
    valve_cols = ['SV1', 'SV2', 'SV3', 'SV4']

    for index, row in df.iterrows():
        cond = row['Condition']

        # LOGICA TASK 1 & 2
        if cond == 'Normal':
            df.at[index, 'Target_Task1'] = 0
            df.at[index, 'Target_Task2'] = 0
        elif cond == 'Anomaly': # Bubble
            df.at[index, 'Target_Task1'] = 1
            df.at[index, 'Target_Task2'] = 2 # 2 = Bubble
        elif cond == 'Fault':   # Valve
            df.at[index, 'Target_Task1'] = 1
            df.at[index, 'Target_Task2'] = 3 # 3 = Valve Fault

        # LOGICA TASK 3 (Bubble)
        for i, col in enumerate(bubble_cols):
            if col in df.columns and row[col] == 'Yes':
                df.at[index, 'Target_Task3'] = i + 1
                break

        # LOGICA TASK 4 & 5 (Valve)
        for i, col in enumerate(valve_cols):
            if col in df.columns:
                val = row[col]
                if isinstance(val, (int, float)) and val < 100:
                    df.at[index, 'Target_Task4'] = i + 1
                    df.at[index, 'Target_Task5'] = val
                    break

    return df

def extract_features(file_path):
    """
    Estrae le features da un singolo file CSV (1200 righe).
    """
    try:
        df = pd.read_csv(file_path)
        feats = {}
        sensors = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']

        # 1. GLOBAL (tutto il file)
        for s in sensors:
            sig = df[s]
            feats[f'{s}_Global_Mean'] = sig.mean()
            feats[f'{s}_Global_Std'] = sig.std()
            feats[f'{s}_Global_Max'] = sig.max()
            feats[f'{s}_Global_Min'] = sig.min()
            feats[f'{s}_Global_PtP'] = sig.max() - sig.min()

        # 2. START (primi 100ms)
        df_start = df[df['TIME'] <= 0.1]
        for s in sensors:
            sig = df_start[s]
            if len(sig) > 0:
                feats[f'{s}_Start_Mean'] = sig.mean()
                feats[f'{s}_Start_Std'] = sig.std()
                feats[f'{s}_Start_Slope'] = sig.iloc[-1] - sig.iloc[0]
            else:
                feats[f'{s}_Start_Mean'] = 0
                feats[f'{s}_Start_Std'] = 0
                feats[f'{s}_Start_Slope'] = 0

        return feats
    except Exception as e:
        print(f"Errore lettura {file_path}: {e}")
        return None

# --- MAIN ESECUZIONE ---
if __name__ == "__main__":

    # 1. Elaboriamo l'Excel
    if os.path.exists(PATH_LABELS):
        df_labels = process_labels_and_targets(PATH_LABELS)
        print(f"Labels caricate correttamente. Totale casi: {len(df_labels)}")
    else:
        print(f"ERRORE: Non trovo il file labels in {PATH_LABELS}")
        exit()

    dataset_rows = []

    print(f"Inizio estrazione feature dai file in: {PATH_TRAIN_DATA}")

    for index, row in df_labels.iterrows():
        case_id = int(row['Case'])

        # FORMATTAZIONE NOME FILE: Case001.csv
        filename = f"Case{case_id:03d}.csv"
        file_path = os.path.join(PATH_TRAIN_DATA, filename)

        if os.path.exists(file_path):
            # 2. Calcoliamo le Features
            features_dict = extract_features(file_path)

            if features_dict:
                # 3. MERGE
                full_row = {**row.to_dict(), **features_dict}
                dataset_rows.append(full_row)
        else:
            # Utile per capire se il percorso è sbagliato
            if index < 3: # Stampiamo l'errore solo per i primi per non intasare
                print(f"File non trovato: {file_path}")

    # 4. Salvataggio su Drive
    if len(dataset_rows) > 0:
        df_final = pd.DataFrame(dataset_rows)

        # Salviamo il file CSV nella stessa cartella 'train' su Drive
        output_filename = os.path.join(BASE_DIR, "dataset_completo_train.csv")
        df_final.to_csv(output_filename, index=False)

        print(f"\nCOMPLETATO!")
        print(f"File salvato in: {output_filename}")
        print(f"Dimensioni finali: {df_final.shape}")
        print("Prime 5 righe delle colonne Target:")
        print(df_final[['Case', 'Target_Task1', 'Target_Task2']].head())
    else:
        print("\nNESSUN DATO ELABORATO. Controlla i percorsi delle cartelle.")

Lettura file labels: /content/drive/MyDrive/Manutenzione/train/labels.xlsx
Labels caricate correttamente. Totale casi: 177
Inizio estrazione feature dai file in: /content/drive/MyDrive/Manutenzione/train/data/

COMPLETATO!
File salvato in: /content/drive/MyDrive/Manutenzione/train/dataset_completo_train.csv
Dimensioni finali: (177, 76)
Prime 5 righe delle colonne Target:
   Case  Target_Task1  Target_Task2
0     1             0             0
1     2             0             0
2     3             0             0
3     4             0             0
4     5             0             0


In [ ]:
 # --- CONFIGURAZIONE PERCORSI TEST ---
BASE_DIR = "/content/drive/MyDrive/test/"
PATH_TEST_DATA = os.path.join(BASE_DIR, "data/")       # Cartella coi CSV grezzi
PATH_TEST_LABELS = os.path.join(BASE_DIR, "labels_spacecraft.xlsx") # File info spacecraft

def extract_features(file_path):
    """
    STESSA IDENTICA FUNZIONE usata per il training.
    È fondamentale che le features siano calcolate allo stesso modo.
    """
    try:
        df = pd.read_csv(file_path)
        feats = {}
        sensors = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']

        # 1. GLOBAL
        for s in sensors:
            sig = df[s]
            feats[f'{s}_Global_Mean'] = sig.mean()
            feats[f'{s}_Global_Std'] = sig.std()
            feats[f'{s}_Global_Max'] = sig.max()
            feats[f'{s}_Global_Min'] = sig.min()
            feats[f'{s}_Global_PtP'] = sig.max() - sig.min()

        # 2. START (primi 100ms)
        df_start = df[df['TIME'] <= 0.1]
        for s in sensors:
            sig = df_start[s]
            if len(sig) > 0:
                feats[f'{s}_Start_Mean'] = sig.mean()
                feats[f'{s}_Start_Std'] = sig.std()
                feats[f'{s}_Start_Slope'] = sig.iloc[-1] - sig.iloc[0]
            else:
                feats[f'{s}_Start_Mean'] = 0
                feats[f'{s}_Start_Std'] = 0
                feats[f'{s}_Start_Slope'] = 0
        return feats
    except Exception as e:
        print(f"Errore {file_path}: {e}")
        return None

# --- MAIN ---
if __name__ == "__main__":
    print(f"Lettura info spacecraft da: {PATH_TEST_LABELS}")

    # Questo file ha solo: Case, Spacecraft
    df_info = pd.read_excel(PATH_TEST_LABELS, header=1) # Modificato: aggiunto header=1

    dataset_rows = []

    print("Inizio elaborazione TEST set...")

    for index, row in df_info.iterrows():
        case_id = int(row['Case#']) # Modificato: cambiato 'ID' in 'Case#'
        # Nota: controlla se nel file excel la colonna è 'ID' o 'Case'.
        # Solitamente nei file labels_spacecraft è 'ID'.

        filename = f"Case{case_id}.csv" # Qui di solito non hanno padding 001, ma controlliamo
        # Se i file si chiamano Case178.csv (senza zeri extra davanti se >100)

        file_path = os.path.join(PATH_TEST_DATA, filename)

        if not os.path.exists(file_path):
             # Proviamo col padding se fallisce (es Case001.. ma qui siamo >100)
             # Di solito nel test set i nomi sono Case178.csv
             pass

        if os.path.exists(file_path):
            feats = extract_features(file_path)
            if feats:
                # Aggiungiamo ID e Spacecraft
                full_row = {
                    'Case': case_id,
                    'Spacecraft': row['Spacecraft#'], # Modificato: cambiato 'Spacecraft' in 'Spacecraft#'
                    **feats
                }
                dataset_rows.append(full_row)
        else:
            print(f"File mancante: {filename}")

    # Salvataggio
    df_final = pd.DataFrame(dataset_rows)
    output_file = os.path.join(BASE_DIR, "dataset_completo_test.csv")
    df_final.to_csv(output_file, index=False)

    print(f"\nGenerazione completata!")
    print(f"File salvato: {output_file}")
    print(f"Dimensioni: {df_final.shape}")

Lettura info spacecraft da: /content/drive/MyDrive/Manutenzione/test/labels_spacecraft.xlsx
Inizio elaborazione TEST set...

Generazione completata!
File salvato: /content/drive/MyDrive/Manutenzione/test/dataset_completo_test.csv
Dimensioni: (46, 58)
